## Implementing Chris Doenlen's 'Bot Or Not' Python Module

Everything from `twitter_funcs.py` was cloned from Chris' [repository](https://github.com/scrapfishies/twitter-bot-detection).

I will use this to label each user as 'bot' (boolean 1/0).

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from twitter_funcs import *

# Imports
import os
import numpy as np
import pandas as pd

import pickle
import tweepy

from datetime import datetime
from secrets import api_secret_key, api_key, bearer_token
import re
import time
import csv

In [2]:
data = pd.read_pickle("pickle/balanced_nov2_tweets.pick")

In [3]:
data.shape

(144000, 9)

In [4]:
data.head(3)

,id,date,time,user_id,username,tweet,hashtags,trump,biden
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,All these articles showing that Biden is in th...,[],False,True
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,[],False,True
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,@The_Grupp “It is purely a fortuity that this ...,[],False,True


I'm getting a rate limit error. according to twitter site, I can lookup 300 users per 15 minutes. Let's try that.

In [6]:
def chunks(user_ids, n):
    """Yield successive n-sized chunks from user_ids (iterable)."""
    lst = list(user_ids)
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    


Here's where we'll implement Chris Doenlen's 'Bot or Not' model.

In [7]:
with open("bot_model.pick", "rb") as read_file:
    xgb_model = pickle.load(read_file)

In [8]:
time.sleep(60*5)

In [ ]:
verifieds = {}
bot_probas = {}

# don't repeat users we already have if script fails
user_ids = list(data.user_id.unique())
try:
    existing_stats = pd.read_csv("../data/user_stats.csv")
    existing_ids = set(existing_stats.user_id.unique())
    for user_id in existing_ids:
        user_ids.remove(existing_id)
    print(len(existing_ids))
except:
    pass

user_id_chunks = list(chunks(user_ids, n=300))

# now get stats for new users

csv_file = open("../data/user_stats.csv", "a")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["user_id", "bot_proba", "verified"])
for chunk in user_id_chunks:
    print(f"Preparing chunk. Num users: {len(chunk)}")
    for user_id in chunk:
        print(f"Preparing user '{user_id}'")
        
        
        auth = tweepy.OAuthHandler(api_key, api_secret_key)
        api = tweepy.API(auth)
        
        try: # Gather features for bot/not bot model
            # Get user information from screen name
            user = api.get_user(user_id)

            # account features to return for predicton
            account_age_days = (datetime.now() - user.created_at).days
            verified = user.verified # will also use this in our data
            geo_enabled = user.geo_enabled
            default_profile = user.default_profile
            default_profile_image = user.default_profile_image
            favourites_count = user.favourites_count
            followers_count = user.followers_count
            friends_count = user.friends_count
            statuses_count = user.statuses_count
            average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

            # manufactured features
            hour_created = int(user.created_at.strftime("%H"))
            network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
            tweet_to_followers = np.round(
                np.log(1 + statuses_count) * np.log(1 + followers_count), 3
            )
            follower_acq_rate = np.round(
                np.log(1 + (followers_count / account_age_days)), 3
            )
            friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

            # organizing list to be returned
            account_features = [
                verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
                followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
                follower_acq_rate,friends_acq_rate]

            if account_features == np.nan:
                proba = np.nan
                verified = np.nan
                csv_writer.writerow([user_id, proba, verified])
                continue

            else:
                user_m = np.matrix(account_features)
                proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
                verified = account_features[0]
                csv_writer.writerow([user_id, proba, verified])

        except:
            print(f'error encountered, skipping user {user_id}')
            proba = np.nan
            verified = np.nan
        
            csv_writer.writerow([user_id, proba, verified])
    print("Chunk complete. Waiting 15 minutes.")
    time.sleep(15*60+1)

csv_file.close()

Preparing chunk. Num users: 300
Preparing user '2820503362'
Preparing user '1312487180258820096'
Preparing user '2335763630'
Preparing user '3282973915'
Preparing user '105937456'
Preparing user '46538055'
Preparing user '100625142'
Preparing user '1219326209366155264'
Preparing user '1254928044529274883'
Preparing user '1288981025364860928'
Preparing user '65067002'
Preparing user '2562614485'
Preparing user '2246207654'
Preparing user '4261171756'
Preparing user '1066144442241953792'
Preparing user '42920728'
Preparing user '35221554'
Preparing user '1007960516'
Preparing user '1118489465008336897'
Preparing user '980467278385958912'
Preparing user '59143598'
Preparing user '1206954619789856769'
Preparing user '1243870145229008896'
Preparing user '1658455681'
Preparing user '989298900'
Preparing user '1267800950321876994'
Preparing user '908298889828106244'
Preparing user '807989050972000256'
Preparing user '849410927158054916'
Preparing user '1090811471985065984'
Preparing user '838

Preparing user '1322777576985735169'
Preparing user '773606728399282176'
Preparing user '44489923'
Preparing user '1291683999392436225'
Preparing user '789191924653498370'
Preparing user '979870450577686528'
Preparing user '1087596246'
Preparing user '1266580535201140736'
Preparing user '1558916702'
Preparing user '45435129'
Preparing user '1216524009728364544'
Preparing user '747223063029227521'
Preparing user '960993253863645184'
Preparing user '3411004725'
Preparing user '35604515'
Preparing user '29460341'
Preparing user '735431735148699649'
Preparing user '2457199196'
Preparing user '591354042'
Preparing user '1068676218516635649'
Preparing user '64909225'
Preparing user '1314028401980846082'
Preparing user '1682291196'
Preparing user '1033807310391394304'
Preparing user '889182369478959105'
Preparing user '2777388739'
Preparing user '1074950605611790336'
Preparing user '1235927786533138433'
Preparing user '930407064'
Preparing user '2708307975'
Preparing user '1299053433257828352

Preparing user '797466618049822722'
Preparing user '1275524470904668160'
Preparing user '337175830'
Preparing user '1090329114827878400'
Preparing user '1023791437417275394'
Preparing user '1122061632224550912'
Preparing user '1220490158656671745'
Preparing user '2161005588'
Preparing user '3911961377'
Preparing user '3124442283'
Preparing user '359512877'
Preparing user '1267950398872133639'
Preparing user '218148636'
Preparing user '24320587'
Preparing user '859109706278858753'
Preparing user '203718874'
Preparing user '17742810'
Preparing user '20442002'
Preparing user '1292619538970939398'
Preparing user '1321966961501315073'
Preparing user '1200973094883676160'
Preparing user '1110239603196256256'
Preparing user '1055551324287369216'
Preparing user '916075489789345792'
Preparing user '1284866944806596608'
Preparing user '1046176148152090624'
Preparing user '19753011'
Preparing user '3073232673'
Preparing user '1320724518155898880'
Preparing user '104334135'
Preparing user '2996493

Preparing user '2617596283'
Preparing user '51776536'
Preparing user '25790437'
Preparing user '315474153'
Preparing user '2475407894'
Preparing user '1254895741690163200'
Preparing user '415142756'
Preparing user '1335059035'
Preparing user '2259260936'
Preparing user '303525188'
Preparing user '978307014605639680'
Preparing user '1270148669371609088'
Preparing user '1460958716'
Preparing user '1548506839'
Preparing user '1076256681334632448'
Preparing user '704481484858658816'
Preparing user '1323229114111844353'
Preparing user '709103640062066688'
Preparing user '1300177071071690753'
Preparing user '1308162986998026240'
Preparing user '4319693115'
Preparing user '2291120303'
Preparing user '863987849921794048'
Preparing user '3991537240'
Preparing user '139596582'
Preparing user '2391934999'
Preparing user '3122854931'
Preparing user '1262830448176234497'
Preparing user '1275811200157257731'
Preparing user '1267828821945794561'
Preparing user '30500949'
Preparing user '1296152764280

Preparing user '223938677'
Preparing user '1259224851518115841'
Preparing user '1211344769877561344'
Preparing user '22327009'
Preparing user '305148929'
Preparing user '1296932965134499840'
Preparing user '2844609570'
Preparing user '700780827928432640'
Preparing user '3378267019'
Preparing user '1912807076'
Preparing user '402965564'
Preparing user '1282129799780036610'
Preparing user '608525749'
Preparing user '960188062348251136'
Preparing user '384376401'
Preparing user '21099562'
Preparing user '1300031675821436928'
Preparing user '1234885878255955968'
Preparing user '1015025453571158016'
Preparing user '1175592411604500480'
Preparing user '1285631360527249408'
Preparing user '1321786666646908930'
Preparing user '279224297'
Preparing user '1309540147713712129'
Preparing user '1104322079585984513'
Preparing user '793473581061550080'
Preparing user '1142068257052139521'
Preparing user '741803'
Preparing user '1021815473116078080'
Preparing user '1305234560909623296'
Preparing user 

...

Preparing user '1211875314621534208'
Preparing user '741376308'
Preparing user '797154744548651008'
Preparing user '427411903'
Preparing user '1042591638'
Preparing user '606146933'
Preparing user '1282027348276248576'
Preparing user '3174843730'
Preparing user '440464127'
Preparing user '15088605'
Preparing user '969740668380749824'
Preparing user '17943624'
Preparing user '2690955980'
Preparing user '23440586'
Preparing user '274274016'
Preparing user '1390597153'
Preparing user '266326338'
Preparing user '164150683'
Preparing user '1074979748'
Preparing user '372529808'
Preparing user '1230875656747208705'
Preparing user '1123369091878641664'
Preparing user '1318953209373839362'
Preparing user '372787472'
Preparing user '3067135782'
Preparing user '378072859'
Preparing user '2859023885'
Preparing user '473476604'
Preparing user '1219356108869001216'
Preparing user '4214519339'
Preparing user '712852680000073728'
Preparing user '964016566730928128'
Preparing user '893529009543426050'

Preparing user '1156275814868168704'
Preparing user '1277610572012498944'
Preparing user '15632337'
Preparing user '1304127684633649156'
Preparing user '788700982221893632'
Preparing user '3138946873'
Preparing user '1927985708'
Preparing user '459302182'
Preparing user '1241040541950803968'
Preparing user '1316890942079528960'
Preparing user '32864795'
Preparing user '1634742997'
Preparing user '1243576765936807937'
Preparing user '772691847802216448'
Preparing user '30837404'
Preparing user '1318234469921361921'
Preparing user '242421463'
Preparing user '1308192476767764480'
Preparing user '4385727183'
Preparing user '1242155368387657728'
Preparing user '842784818'
Preparing user '869785084479631360'
Preparing user '2334210600'
Preparing user '801303877'
Preparing user '1196213158433771523'
Preparing user '3771043037'
Preparing user '903313767596822528'
Preparing user '4861611379'
Preparing user '1317879361190694912'
Preparing user '248469208'
Preparing user '142096963'
Preparing use

Preparing user '1705391150'
Preparing user '1273505240050855938'
Preparing user '556358153'
Chunk complete. Waiting 15 minutes.
Preparing chunk. Num users: 300
Preparing user '1159570378844323840'
Preparing user '979057500212400128'
Preparing user '1858435243'
Preparing user '384800892'
Preparing user '89535688'
Preparing user '630252907'
Preparing user '230796821'
Preparing user '46137585'
Preparing user '1281268155101392898'
Preparing user '1249270652156219392'
Preparing user '746034707209527296'
Preparing user '1129162895647023109'
Preparing user '310546956'
Preparing user '824808971261378560'
Preparing user '3308592113'
Preparing user '2810460354'
Preparing user '1272770048'
Preparing user '1317680521250435072'
Preparing user '759198419739090944'
Preparing user '201963182'
Preparing user '1322054991528546304'
Preparing user '1901740502'
Preparing user '1375536284'
Preparing user '1042763115207778304'
Preparing user '1316214703496941568'
Preparing user '73899524'
Preparing user '131

Preparing user '1233031329597599744'
Preparing user '2392247028'
Preparing user '1197177917211578370'
Preparing user '556480811'
Preparing user '823924739051835393'
Preparing user '1246202769662291969'
Preparing user '1084896359721910272'
Preparing user '703503924'
Preparing user '622286080'
Preparing user '1237385648035614720'
Preparing user '1322637393191669760'
Preparing user '2389419156'
Preparing user '969635344462344194'
Preparing user '23023965'
Preparing user '234773300'
Preparing user '1287969203748732931'
Preparing user '305291107'
Preparing user '1091141124574638080'
Preparing user '1257051224'
Preparing user '1207435373850189826'
Preparing user '1186275111592222723'
Preparing user '3871502234'
Preparing user '1089937044435030016'
Preparing user '592945059'
Preparing user '1248018965881896961'
Preparing user '879521086534189056'
Preparing user '4190667705'
Preparing user '2340995202'
Preparing user '1245057759768444938'
Preparing user '3193046374'
Preparing user '3325778140'

Preparing user '920087392048046080'
Preparing user '327491528'
Preparing user '1261681896506880000'
Preparing user '564044174'
Preparing user '2838159825'
Preparing user '64829826'
Preparing user '1061989359744925697'
Preparing user '701815183170113536'
Preparing user '1224546660791017472'
Preparing user '414010452'
Preparing user '792450594661404672'
Preparing user '3680034562'
Preparing user '36569231'
Preparing user '18272274'
Preparing user '605415583'
Preparing user '785284455770497024'
Preparing user '1163458704915730432'
Preparing user '368778613'
Preparing user '136717052'
Preparing user '1316111836664737793'
Preparing user '839962012964835328'
Preparing user '462451128'
Preparing user '291935994'
Preparing user '4031877857'
Preparing user '1216574102720385024'
Preparing user '75852634'
Preparing user '1066346133386141698'
Preparing user '111486145'
Preparing user '1279586549991198720'
Preparing user '1298963325078900736'
Preparing user '883343376187351040'
Preparing user '3008

In [ ]:
csv_file.close()

In [ ]:
bot_probas

In [ ]:
data['bot_proba'] = data['user_id'].map(bot_probas)
data['verifieds'] = data['user_id'].map(is_verified)
data.to_pickle("pickle/tw_proba_verif.pick")

In [ ]:
verifieds

In [ ]:
data['verified'] = data['user_id'].map(verifieds)
data['bot_proba'] = data['user_id'].map(bot_probas)
